In [1]:
import emat

Load a scope from a yaml file.

In [2]:
with open('gbnrtc_scope.yaml') as yf:
    for _ in range(100):
        print(yf.readline(), end="")

---
# EMAT Scope Definition
# 

scope:
    name: GBNRTC
    desc: EMAT Prototype using TransCAD

inputs:

    ## UNCERTAINTIES ##

    Land Use - CBD Focus:
        ptype: exogenous uncertainty
        desc: Change in overall land use with greatest effect in CBD
        default: 1.0
        min: 0.82
        max: 1.37
        dist:
            name: pert
            rel_peak: 0.33
            gamma: 4
        corr: []

    Freeway Capacity:
        ptype: exogenous uncertainty
        desc: Change in freeway capacity due to Vehicle Technology
        default: 1.0
        min: 1.0
        max: 2.0
        dist:
            name: triangle
            rel_peak: 0.5
        corr: []
        
    Auto IVTT Sensitivity:
        ptype: exogenous uncertainty
        desc: Change in sensitivity to Auto travel time due to Vehicle Technology
        default: 1.0
        min: 0.75
        max: 1.0
        dist:
            name: triangle
            rel_peak: 0.5
        corr: []
        
    Shar

In [3]:
scope = emat.Scope('gbnrtc_scope.yaml')

In [4]:
scope

<emat.Scope with 4 uncertainties, 2 levers, 46 measures>

Create a brand new set of `Boxes`.

In [5]:
boxes = emat.Boxes(scope=scope)

Define a new top-level box (i.e. a box with no parents to inherit from).

In [6]:
box_1 = emat.Box(name='High Population Growth', scope=scope)

Add a lower bound for population growth, to ensure the include values are all "high".

In [7]:
box_1.set_lower_bound('Land Use - CBD Focus', 1.0)

Add some things as "relevant features" that we are interested in analyzing,
even though we do not set bounds on them (yet).

In [8]:
box_1.relevant_features.add('Total LRT Boardings')
box_1.relevant_features.add('Downtown to Airport Travel Time')
box_1.relevant_features.add('Peak Transit Share')
box_1.relevant_features.add('AM Trip Time (minutes)')
box_1.relevant_features.add('AM Trip Length (miles)')
box_1.relevant_features.add('Region-wide VMT')
box_1.relevant_features.add('Total Transit Boardings')
box_1.relevant_features.add('Corridor Kensington Daily VMT')

In [9]:
box_1

Box: High Population Growth
   ●             Land Use - CBD Focus >= 1.0
   ◌                  Region-wide VMT
   ◌          Total Transit Boardings
   ◌  Downtown to Airport Travel Time
   ◌               Peak Transit Share
   ◌              Total LRT Boardings
   ◌    Corridor Kensington Daily VMT
   ◌           AM Trip Length (miles)
   ◌           AM Trip Time (minutes)

Define a new lower level box, which will inherit from the top level box we just created.

In [10]:
box_2 = emat.Box(name='Automated Vehicles', scope=scope, parent='High Population Growth')

Set some thresholds on this lower level box.

In [11]:
box_2.set_lower_bound('Freeway Capacity', 1.25)
box_2.set_upper_bound('Auto IVTT Sensitivity', 0.9)

In [12]:
box_2

Box: Automated Vehicles
   ●       Freeway Capacity >= 1.25
   ●  Auto IVTT Sensitivity <= 0.9

In [13]:
box_2.parent_box_name

'High Population Growth'

So far, the individual boxes we created are just loose boxes.  To connect them, we need to add them to the master `Boxes` object.

In [14]:
boxes.add(box_1)
boxes.add(box_2)

We can check on what named boxes are in a `Boxes` object with the `plain_names` method,
which just gives the names, or the `fancy_names` method, which adds some icons
to help indicate the hierarchy.

In [15]:
boxes.plain_names()

[None, 'High Population Growth', 'Automated Vehicles']

In [16]:
boxes.fancy_names()

['Scope: GBNRTC', '▶ High Population Growth', '▷ ▶ Automated Vehicles']

Now that the boxes are linked together in a `Boxes` object, we can use the `get_chain` method to aggregate
the attributes of any box along with all parents in the chain.

In [17]:
boxes.get_chain('Automated Vehicles')

ChainedBox: Automated Vehicles
   ●             Land Use - CBD Focus >= 1.0
   ●                 Freeway Capacity >= 1.25
   ●            Auto IVTT Sensitivity <= 0.9
   ◌                  Region-wide VMT
   ◌          Total Transit Boardings
   ◌  Downtown to Airport Travel Time
   ◌               Peak Transit Share
   ◌              Total LRT Boardings
   ◌    Corridor Kensington Daily VMT
   ◌           AM Trip Length (miles)
   ◌           AM Trip Time (minutes)